# Optimization Examples

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/slickml/slick-ml/blob/master/examples/optimization.ipynb)

### Google Colab Configuration

In [ ]:
!git clone https://github.com/slickml/slick-ml.git
%cd slick-ml
!pip install -r requirements.txt

### Local Environment Configuration

In [1]:
# # Change path to project root
%cd ..

/Users/tracesmith/Desktop/Trace/Code_Library/slick-ml


### Import Python Libraries

In [2]:
%load_ext autoreload

# widen the screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# change the path and loading class
import os, sys
import pandas as pd
import numpy as np
import seaborn as sns

In [73]:
%autoreload
from slickml.optimization import XGBoostClassifierBayesianOpt

______
# Optimization

#### Load Data + Split Train/Test

In [3]:
# loading data
df = pd.read_csv("data/clf_data.csv")
df.head()

,F1,F2,F3,F4,F5,F6,CLASS
0,5.7,4.7,3.7,58.0,249.30,456.47,0
1,7.7,6.6,4.1,20.0,260.92,443.43,1
2,6.2,4.3,4.6,59.0,255.63,478.96,1
3,5.7,4.4,3.8,49.0,195.28,381.94,0
4,9.1,4.7,4.6,17.0,259.55,395.67,1


In [4]:
# define X, y
y = df.CLASS.values
X = df.drop(["CLASS"], axis=1)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    stratify=y,
                                                    random_state=1367)

# XGBoostClassifierBayesianOpt

In [7]:
# define bayesian optimizer 
xbo = XGBoostClassifierBayesianOpt()

In [8]:
# train the optimizer on train set
xbo.fit(X_train, y_train)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.8245   |  0.8975   |  0.04571  |  0.6628   |  4.238    |  1.436    |  0.3064   |  0.7136   |  0.1931   |
|  2        |  0.8374   |  0.7904   |  0.6447   |  0.9152   |  3.334    |  3.238    |  0.7772   |  0.269    |  0.9726   |
|  3        |  0.8199   |  0.8498   |  0.6044   |  0.6874   |  6.651    |  15.7     |  0.061    |  0.5114   |  0.6848   |
|  4        |  0.8551   |  0.7297   |  0.8513   |  0.4627   |  4.757    |  4.965    |  0.9328   |  0.363    |  0.9365   |
|  5        |  0.8173   |  0.5425   |  0.5451   |  0.8782   |  6.633    |  5.028    |  0.1845   |  0.333    |  0.9125   |
|  6        |  0.8099   |  0.4336   |  0.282    |  0.3017   |  4.288    |  16.0     |  0.06196  |  0.8237   |  0.8923   |
|  7        |  0.8386   

In [9]:
# optimization results (or xbo.optimization_results_)
xbo.get_optimization_results()

,colsample_bytree,gamma,learning_rate,max_depth,min_child_weight,reg_alpha,reg_lambda,subsample,auc
0,0.897530,0.045712,0.662807,4.238468,1.435660,0.306424,0.713585,0.193055,0.824512
1,0.790404,0.644709,0.915190,3.334492,3.238280,0.777161,0.269010,0.972576,0.837449
2,0.849819,0.604370,0.687435,6.651023,15.698338,0.061001,0.511379,0.684811,0.819881
3,0.729727,0.851274,0.462704,4.756996,4.964748,0.932765,0.362983,0.936539,0.855133
4,0.542456,0.545092,0.878165,6.632704,5.028311,0.184497,0.333049,0.912511,0.817250
5,0.433647,0.281979,0.301710,4.287809,15.997975,0.061965,0.823726,0.892268,0.809917
6,0.527662,0.710147,0.316749,2.901123,9.435776,0.691293,0.125812,0.646834,0.838585
7,0.129065,0.224998,0.896721,6.215583,3.885759,0.509329,0.256423,0.391094,0.807489
8,0.784692,0.884516,0.463588,4.454321,4.869008,0.969036,0.362743,0.979997,0.854646
9,0.821392,1.000000,0.231482,4.570813,5.632603,1.000000,0.394688,1.000000,0.864417


In [10]:
# best performance (or xbo.best_performance_)
xbo.get_best_performance()

,colsample_bytree,gamma,learning_rate,max_depth,min_child_weight,reg_alpha,reg_lambda,subsample,auc
0,0.821392,1.0,0.231482,4.570813,5.632603,1.0,0.394688,1.0,0.864417


In [11]:
# tuned params (or xbo.best_params_)
xbo.get_best_params()

{'colsample_bytree': 0.8213916662259918,
 'gamma': 1.0,
 'learning_rate': 0.23148232373451072,
 'max_depth': 4,
 'min_child_weight': 5.632602921054691,
 'reg_alpha': 1.0,
 'reg_lambda': 0.39468801734425263,
 'subsample': 1.0}

In [12]:
# optimizer object (or xbo.optimizer_)
xbo.get_optimizer()

In [13]:
# optimization params boundaries 
xbo.get_pbounds()

{'max_depth': (2, 7),
 'learning_rate': (0, 1),
 'min_child_weight': (1, 20),
 'colsample_bytree': (0.1, 1.0),
 'subsample': (0.1, 1),
 'gamma': (0, 1),
 'reg_alpha': (0, 1),
 'reg_lambda': (0, 1)}

In [14]:
xbo.pbounds

{'max_depth': (2, 7),
 'learning_rate': (0, 1),
 'min_child_weight': (1, 20),
 'colsample_bytree': (0.1, 1.0),
 'subsample': (0.1, 1),
 'gamma': (0, 1),
 'reg_alpha': (0, 1),
 'reg_lambda': (0, 1)}

______
# XGBoostClassifierHyperOpt

In [30]:
from hyperopt import hp
def get_xgb_params():
    """ Define Parameter Space"""
    params  = {'nthread':4,
              'booster':'gbtree',
              'tree_method':'hist',
              'objective':'binary:logistic',
              'max_depth':hp.choice('max_depth',range(5,100,5)),
              'learning_rate':hp.quniform('learning_rate',0.01,0.50,0.01),
              'gamma':hp.quniform('gamma',0,0.50,0.01),
              'min_child_weight':hp.quniform('min_child_weight',1,10,1),
              'subsample':hp.quniform('subsample',0.1,1,0.01),
              'colsample_bytree': hp.quniform('colsample_bytree',0.1,1.0,0.01),
              'gamma': hp.quniform('gamma',0.0,1.0,0.01),
              'reg_alpha':hp.quniform('reg_alpha',0.0,1.0,0.01),
              'reg_lambda':hp.quniform('reg_lambda',0.0,1.0,0.01)
              }
    return params

In [44]:
%autoreload
from slickml.classification import XGBoostClassifier
from slickml.optimization import XGBoostClassifierHyperOpt

xgbh = XGBoostClassifierHyperOpt(num_boost_rounds=200,
                                 metrics='logloss',
                                 n_split=3,
                                 shuffle=True,
                                 early_stop=20,
                                 func_name='xgb_cv',
                                 space=get_xgb_params(),
                                 max_evals=100,
                                 verbose=False
                                )

xgbh.fit(X_train,y_train)

100%|██████████| 100/100 [00:04<00:00, 21.88trial/s, best loss: 0.312143]


### Best set of parameters from all runs

In [45]:
xgbh.get_optimization_results()

,colsample_bytree,gamma,learning_rate,max_depth,min_child_weight,reg_alpha,reg_lambda,subsample
0,0.58,0.14,0.48,17,2.0,0.31,0.66,0.77


### Results from each trial

In [51]:
import pprint
pprint.pprint(xgbh.get_optimization_trials().trials)

[{'book_time': datetime.datetime(2020, 10, 3, 18, 30, 33, 75000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
           'idxs': {'colsample_bytree': [0],
                    'gamma': [0],
                    'learning_rate': [0],
                    'max_depth': [0],
                    'min_child_weight': [0],
                    'reg_alpha': [0],
                    'reg_lambda': [0],
                    'subsample': [0]},
           'tid': 0,
           'vals': {'colsample_bytree': [0.97],
                    'gamma': [0.4],
                    'learning_rate': [0.09],
                    'max_depth': [15],
                    'min_child_weight': [3.0],
                    'reg_alpha': [0.11],
                    'reg_lambda': [0.71],
                    'subsample': [0.64]},
           'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2020, 10, 3, 18, 30, 33, 87000),
  'result': {'loss': 0.38027000000000005, 'status': 'ok'},
